In [ ]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import csv
import json
import btalib
from binance.enums import *
from binance.exceptions import BinanceAPIException, BinanceOrderException

In [ ]:
#Connecting API client to api account
client = Client(api_key, api_secret)
price = {'BTCUSDT': None, 'error': False}

In [ ]:
#Printing account information
client.get_account()

In [ ]:
# get balance for a specific asset only (BTC)
print(client.get_asset_balance(asset='BTC'))

In [ ]:
# get latest price from Binance API
btc_price = client.get_symbol_ticker(symbol="BTCUSDT")
# print full output (dictionary)
print(btc_price['price'])

In [ ]:
from time import sleep

In [ ]:
#Creating a dictionary to hold our price and let us know of a problem with the web socket
btc_price = {'error':False}

In [ ]:
#Function telling the web socket what to do with the data
def btc_trade_history(msg):
    ''' define how to process incoming WebSocket messages '''
    if msg['e'] != 'error':
        print(msg['c'])
        btc_price['last'] = msg['c']
        btc_price['bid'] = msg['b']
        btc_price['last'] = msg['a']
        btc_price['error'] = False
    else:
        btc_price['error'] = True

In [ ]:
# init and start the WebSocket
bsm = ThreadedWebsocketManager()


In [ ]:
# subscribe to a stream
bsm.start_symbol_ticker_socket(callback=btc_trade_history, symbol='BTCUSDT')
bsm.start_symbol_ticker_socket(callback=btc_trade_history, symbol='ETHUSDT')

In [ ]:
# stop websocket
bsm.stop()

In [ ]:
#Full data info
help(ThreadedWebsocketManager)

In [ ]:
# valid intervals - 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M

# get timestamp of earliest date data available\converted from Unix time using Pandas
timestamp = client._get_earliest_valid_timestamp('BTCUSDT', '1d')
print(pd.to_datetime(timestamp, unit='ms'))

In [ ]:
# request historical candle (or klines) data
bars = pd.DataFrame(client.get_historical_klines('BTCUSDT', '1d', timestamp, limit=1000))
bars = bars.iloc[:,:5]
bars.columns=['date', 'open', 'high', 'low', 'close']


In [ ]:
# option 1 - save to file using json method
with open('btc_bars.json', 'w') as e:
    json.dump(bars, e)

In [ ]:
# option 2 - save as CSV file using the csv writer library
with open('btc_bars.csv', 'w', newline='') as f:
        wr = csv.writer(f)
    for line in bars:
        wr.writerow(line)

In [ ]:
# option 3 - save as CSV file without using a library(allows to choose columns as you save)
with open('btc_bars2.csv', 'w') as d:
    for line in bars:
        d.write(f'{line[0]}, {line[1]}, {line[2]}, {line[3]}, {line[4]}\n')

In [ ]:
# option 4 - create a Pandas DataFrame and export to CSV(best if planning on holding data in a DataFrame)
btc_df = pd.DataFrame(bars)
btc_df.set_index('date', inplace=True)
btc_df.index = pd.to_datetime(btc_df.index, unit='ms')
print(btc_df.head())

In [ ]:
# export DataFrame to csv
btc_df.to_csv('btc_bars3.csv')

In [ ]:
# load DataFrame
btc_df = pd.read_csv('btc_bars3.csv', index_col=0)

# calculate 20 moving average using Pandas
btc_df['20 Day MA'] = btc_df.close.rolling(20).mean()
print(btc_df.tail(5))

In [ ]:
# calculate just the last value for the 20 moving average
mean = btc_df.close.tail(20).mean()
mean

In [ ]:
#Using Bta-library to calculate moving averages(30 day moving average default). returns type=object(use .df to output)
sma = btalib.sma(btc_df.close)
print(sma.df)


In [ ]:
# create sma and attach as column to original df
btc_df['sma'] = btalib.sma(btc_df.close, period=20).df
print(btc_df.tail())

In [ ]:
#Calculating RSI using Bta-library, returns type=object(use .df to output)
rsi = btalib.rsi(btc_df, period=14)
rsi.df
#We can print the last value with this command
print(rsi.df.rsi[-1])


In [ ]:
#Calculating MACD using Bta-library. Returns type=Object(use .df to output)
macd = btalib.macd(btc_df, pfast=20, pslow=50, psignal=13)
macd.df

In [ ]:
# join the rsi and macd calculations as columns in original df
btc_df = btc_df.join([rsi.df, macd.df])
print(btc_df.tail())

In [ ]:
#Create a Limit Sell order
sell_order_limit = client.create_order(
    symbol='BTCUSDT',
    side='SELL',
    type='LIMIT',
    timeInForce='GTC',
    quantity=0.001,
    price=50000)

In [ ]:
#Creating an order with an Exception block(Order confirmation stored in Variable)

try:
    sell_limit = client.create_order(
        symbol='BTCUSDT',
        side='SELL',
        type='LIMIT',
        timeInForce='GTC',
        quantity=0.001,
        price=50000)

except BinanceAPIException as e:
    # error handling goes here
    print(e)
except BinanceOrderException as e:
    # error handling goes here
    print(e)

In [ ]:
#Checking order confirmation(Dictionary Format)
sell_limit

In [ ]:
#cancel previous orders using orderID(Confirmation stored in Variable)
cancel = client.cancel_order(symbol='BTCUSDT', orderId=sell_limit['orderId'])

In [ ]:
#checking cancel confirmation
cancel

In [ ]:
#Limit Order using a helper function
sell_limit = client.order_limit_sell(symbol='BTCUSDT', quantity=0.001, price=50000)

#market order using a helper function
#market_order = client.order_market_sell(symbol='ETHUSDT', quantity=100)

In [ ]:
# Helper-Function
#     order_limit_buy()
#     order_limit_sell()
#     order_market_buy()
#     order_market_sell()
#     order_oco_buy()
#     order_ocosell()

In [ ]:
#Implementing a stop loss order
try:
    order = client.create_oco_order(
        symbol='BTCUSDT',
        side='SELL',
        quantity=0.001,
        price=30000,
        stopPrice=28500,
        stopLimitPrice=28500,
        stopLimitTimeInForce='GTC')

except BinanceAPIException as e:
    # error handling goes here
    print(e)
except BinanceOrderException as e:
    # error handling goes here
    print(e)

In [ ]:
# use exchange info to confirm order types
info = client.get_symbol_info('ETHUSDT')
print(info['orderTypes'])

In [ ]:
#Purchasing ETH when BTC Goes higher than 50,000 Dollars
bsm = ThreadedWebsocketManager()
bsm.start()
bsm.start_symbol_ticker_socket(symbol='BTCUSDT', callback=btc_pairs_trade)
price = {'BTCUSDT': None, 'error': False}


#Callback function where the Websocket data flows through
def btc_pairs_trade(msg):
    ''' define how to process incoming WebSocket messages '''
    if msg['e'] != 'error':
        price['BTCUSDT'] = float(msg['c'])
    else:
        price['error'] = True


while True:
    # error check to make sure WebSocket is working
    if price['error']:
        # stop and restart socket
        bsm.stop()
        sleep(2)
        bsm.start()
        price['error'] = False

    else:
        if price['BTCUSDT'] > 50000:
            try:
                order = client.order_market_buy(symbol='ETHUSDT', quantity=100)
                break
            except Exception as e:
                print(e)

    sleep(0.1)

In [ ]:
bsm.stop()

In [ ]:
#Executing ETH trade when BTC moves 5% in last 5 min
price = {'BTCUSDT': pd.DataFrame(columns=['date', 'price']), 'error': False}
#Call back function
def btc_pairs_trade(msg):
    ''' define how to process incoming WebSocket messages '''
    if msg['e'] != 'error':
        price['BTCUSDT'].loc[len(price['BTCUSDT'])] = [pd.Timestamp.now(), float(msg['c'])]
    else:
        price['error'] = True
        
# init and start the WebSocket
bsm = ThreadedWebsocketManager()
bsm.start()
bsm.start_symbol_ticker_socket(symbol='BTCUSDT', callback=btc_pairs_trade)

## main
while len(price['BTCUSDT']) == 0:
        # wait for WebSocket to start streaming data
        sleep(0.1)
        
sleep(300)

while True:
        # error check to make sure WebSocket is working
    if price['error']:
        # stop and restart socket
        bsm.stop()
        sleep(2)
        bsm.start()
        price['error'] = False
    else:
                df = price['BTCUSDT']
                start_time = df.date.iloc[-1] - pd.Timedelta(minutes=5)
                df = df.loc[df.date >= start_time]
                max_price = df.price.max()
                min_price = df.price.min()
                
if df.price.iloc[-1] < max_price * 0.95:
                        try:
                                order = client.futures_create_order(symbol='ETHUSDT', side='SELL', type='MARKET', quantity=100)
                                break
                        except Exception as e:
                                print(e)

                elif df.price.iloc[-1] > min_price * 1.05:
                        try:
                                order = client.futures_create_order(symbol='ETHUSDT', side='BUY', type='MARKET', quantity=100)
                                break
                        except Exception as e:
                                print(e)

	sleep(0.1)

In [ ]:
bsm.stop()